# Video pipeline with Yolo + Resnet Model

In [1]:
import os
import sys
from pprint import PrettyPrinter
import matplotlib.pyplot as plt
from typing import Dict, List
import numpy as np

pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
__file__ = globals()["_dh"][0]
project_dir = __file__ = globals()["_dh"][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, "..", "..", "..")))

from experiments.utils.constants import FINAL_RESULTS_PATH
from experiments.utils.parser import AdaptationParser
import experiments.utils.drawing

In [2]:
metaseries = 14

series_meta = {
    "video": {"IPA": 1, "FA2-low": 2, "FA2-high": 3, "RIM": 5},
    "audio-qa": {"IPA": 21, "FA2-low": 22, "FA2-high": 23, "RIM": 25},
    "audio-sent": {"video": 41, "audio-qa": 42, "audio-sent": 43, "sum-qa": 45},
    "sum-qa": {"video": 61, "audio-qa": 62, "audio-sent": 63, "sum-qa": 65}
}

serie_pipeline = {}
for pipeline, series in series_meta.items():
    for serie_name, serie in series.items():
        serie_pipeline[serie] = pipeline
        
        
series_paths = {
series: os.path.join(FINAL_RESULTS_PATH, "metaseries", str(metaseries), "series", str(series))
    for series in serie_pipeline.keys()
}

loaders = {
series: AdaptationParser(
        series_path=series_path, model_name="video", type_of="router_pipeline"
    )
    for series, series_path in series_paths.items()
}

In [3]:
loaders

{1: <experiments.utils.parser.AdaptationParser at 0x7fec3373ddf0>,
 2: <experiments.utils.parser.AdaptationParser at 0x7fed044e8c40>,
 3: <experiments.utils.parser.AdaptationParser at 0x7fec3e4b1dc0>,
 5: <experiments.utils.parser.AdaptationParser at 0x7fed045724c0>,
 21: <experiments.utils.parser.AdaptationParser at 0x7fed04572730>,
 22: <experiments.utils.parser.AdaptationParser at 0x7fed0459b7f0>,
 23: <experiments.utils.parser.AdaptationParser at 0x7fed0459b640>,
 25: <experiments.utils.parser.AdaptationParser at 0x7fed0455e340>,
 41: <experiments.utils.parser.AdaptationParser at 0x7fec3349b3a0>,
 42: <experiments.utils.parser.AdaptationParser at 0x7fec334a61c0>,
 43: <experiments.utils.parser.AdaptationParser at 0x7fec3347cb50>,
 45: <experiments.utils.parser.AdaptationParser at 0x7fec3373dca0>,
 61: <experiments.utils.parser.AdaptationParser at 0x7fec3373dd30>,
 62: <experiments.utils.parser.AdaptationParser at 0x7fec3373dd60>,
 63: <experiments.utils.parser.AdaptationParser at 0

# Real Latencies

In [4]:
timeout_per_second = {}
per_second_results = {}
for series in serie_pipeline.keys():
    timeout_per_second[series], per_second_results[series] = loaders[
        series
    ].per_second_result_processing()

In [5]:
metric = "p99"  # [min, max, p99]
metrics_all = {}
for series in serie_pipeline.keys():
    metric_columns = list(
        filter(lambda col: metric in col, per_second_results[series].columns)
    )
    metrics_all[series] = per_second_results[series][metric_columns]
    metrics_all[series] = metrics_all[series].to_dict(orient="list")
ylabel = "Second"
data = {}


for serie, pipeline in serie_pipeline.items():
    if data.get(pipeline) is None:
        data[pipeline] = {}
    data[pipeline][serie] = metrics_all[serie]["e2e_p99"]

print(data.keys())

dict_keys(['video', 'audio-qa', 'audio-sent', 'sum-qa'])


In [6]:
import importlib

importlib.reload(experiments.utils.drawing)


# experiments.utils.drawing.draw_cdf(data, 10, vertical_label="SLA", bbox_to_anchor=(1.1, 1.5))

<module 'experiments.utils.drawing' from '/home/cc/infernece-pipeline-joint-optimization/experiments/utils/drawing.py'>